# Promoting Belonging in r/feminism
J. Nathan Matias, October 2019

Generating dataset for analysis. Pre-analysis plan at [osf.io/xu258/](https://osf.io/xu258/).

In [5]:
%matplotlib inline
import inspect, os, sys, copy, pytz, re, glob, csv, uuid
os.environ['AIRBRAKE_API_KEY'] = "" ## EDIT BEFORER USING
os.environ['AIRBRAKE_PROJECT_ID'] = "" ## EDIT BEFORE USING

import simplejson as json
import pandas as pd
from dateutil import parser
import datetime
import matplotlib.pyplot as plt   # Matplotlib for plotting
import matplotlib.dates as md
import numpy as np
import seaborn as sns
from collections import Counter, defaultdict
utc=pytz.UTC

ENV = "production"
os.environ['CS_ENV'] = 'production'
BASE_DIR = "/usr/local/civilservant/platform"
sys.path.append(BASE_DIR)

with open(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV), "r") as config:
  DBCONFIG = json.loads(config.read())

### LOAD SQLALCHEMY
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text, and_, or_
import sqlalchemy.orm.session
import utils.common


db_engine = create_engine("mysql://{user}:{password}@{host}/{database}".format(
    host = DBCONFIG['host'],
    user = DBCONFIG['user'],
    password = DBCONFIG['password'],
    database = DBCONFIG['database']))
DBSession = sessionmaker(bind=db_engine)
db_session = DBSession()


### LOAD PRAW
#import reddit.connection
#conn = reddit.connection.Connect(base_dir=BASE_DIR, env="jupyter")

from app.models import *

### FILTER OUT DEPRECATION WARNINGS ASSOCIATED WITH DECORATORS
# https://github.com/ipython/ipython/issues/9242
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')

Logging to /usr/local/civilservant/platform/logs/CivilServant_production.log


# Load Experiment

In [6]:
def return_experiment_objects(experiment_name):
    experiment_obj = db_session.query(Experiment).filter(
                   Experiment.name == experiment_name
                 ).first()
    subreddit_id = json.loads(experiment_obj.settings_json)['subreddit_id']
    participants = db_session.query(ExperimentThing).filter(
                   ExperimentThing.experiment_id == experiment_obj.id
               ).all()
    usernames = [x.thing_id for x in participants]
    participant_comments = db_session.query(Comment).filter(and_(
                                Comment.user_id.in_(usernames),
                                Comment.subreddit_id == subreddit_id
    )).all()

    actions = db_session.query(ExperimentAction).filter(
                                ExperimentAction.experiment_id == experiment_obj.id).all()
    
    return {
        "participants": participants,
        "usernames": usernames,
        "participant_comments": participant_comments,
        "actions": actions,
        "subreddit_id": subreddit_id
    }

In [7]:
first_round_results = return_experiment_objects("newcomer_messaging_experiment-feminism-07.2018")
second_round_results = return_experiment_objects("newcomer_messaging_experiment-feminism-01.2020")

In [8]:
participants = first_round_results['participants'] + second_round_results['participants']
usernames = list(set(first_round_results['usernames'] + second_round_results['usernames']))
participant_comments = first_round_results['participant_comments'] + second_round_results['participant_comments']
actions = first_round_results['actions'] + second_round_results['actions']

In [9]:
participant_modlog = db_session.query(ModAction).filter(and_(
                            ModAction.target_author.in_(usernames),
                            ModAction.subreddit_id == first_round_results['subreddit_id']
)).order_by(ModAction.created_utc.asc()).all()

In [10]:
def participant_object():
    return {
        "username": None,
        "treatment": None,
        "comment_id": None,
        "submission_id": None,
        "message_status": None,
        "randomization": None,
        "block_id": None,
        "assignment_datetime": None,
        "comments": [],
        "ban_actions": [],
        "comments_2_weeks": None,
        "comments_4_weeks": None,
        "comments_8_weeks": None,
        "ban_days_2_weeks": 0,
        "ban_days_4_weeks": 0,
        "ban_days_8_weeks": 0
    }

# Create Dict of Participants 

In [11]:
## key is username
units = defaultdict(participant_object)
for participant in participants:
    metadata = json.loads(participant.metadata_json)
    randomization = metadata['randomization']
    units[participant.thing_id]['block_id']      = randomization["block.id"]
    units[participant.thing_id]['randomization'] = randomization[""]
    units[participant.thing_id]['treatment']     = int(randomization['treatment'])
    units[participant.thing_id]['comment_id']    = metadata["comment_id"]
    units[participant.thing_id]['submission_id'] = metadata["submission_id"]
    units[participant.thing_id]['assignment_datetime'] = participant.created_at
    units[participant.thing_id]['message_status'] = metadata['message_status']

In [12]:
participant_comments = sorted(participant_comments, key = lambda x: x.created_utc)
for comment in participant_comments:
    user_id = comment.user_id
    units[user_id]['comments'].append(comment)

# Create and Validate Experiment Count Outcomes from Dict of Participants

In [13]:
#Counter([len(x['comments']) for x in units.values()])
for unit in units.values():
    two_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*2)
    three_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*3)
    four_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*4)
    five_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*5)
    six_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*6)
    seven_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*7)
    eight_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*8)
    nine_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*9)
    ten_weeks = unit['assignment_datetime'] + datetime.timedelta(days=7*10)
    unit['comments_2_weeks'] = len([x for x in unit['comments'] if x.created_utc < two_weeks])
    unit['comments_3_weeks'] = len([x for x in unit['comments'] if x.created_utc < three_weeks])
    unit['comments_4_weeks'] = len([x for x in unit['comments'] if x.created_utc < four_weeks])
    unit['comments_5_weeks'] = len([x for x in unit['comments'] if x.created_utc < five_weeks])
    unit['comments_6_weeks'] = len([x for x in unit['comments'] if x.created_utc < six_weeks])
    unit['comments_7_weeks'] = len([x for x in unit['comments'] if x.created_utc < seven_weeks])
    unit['comments_8_weeks'] = len([x for x in unit['comments'] if x.created_utc < eight_weeks])
    unit['comments_9_weeks'] = len([x for x in unit['comments'] if x.created_utc < nine_weeks]) 
    unit['comments_10_weeks'] = len([x for x in unit['comments'] if x.created_utc < ten_weeks])   

# Load and Merge Survey Results

In [16]:
data_dir = "" ## UPDATE BEFORE RUNNING
survey_rows = {}

with open(os.path.join(data_dir, "feminism-post-survey-downloaded-05.12.2020.csv")) as f:
    for row in csv.DictReader(f):
        survey_rows[row['the reddit username you used to comment in r/feminism']] = row

In [17]:
survey_colnames = {
   "Did you identify as a feminist at the time you made your first comment in r/feminism?": "identify_feminist",
   'Select which number, corresponding to the images above, best describes your relationship with r/feminism.': "community_closeness",
   'If you have any comments or thoughts related to the sub, you may share them with us here:': "comments",
   "Timestamp": "timestamp"
}

### How Many Surveys Overlap with Participants in the Survey?

In [18]:
print("{0} total surveys".format(len(list(survey_rows.keys()))))

matched_usernames = 0
for username in set(survey_rows.keys()):
    if username in list(units.keys()):
        matched_usernames += 1

print("{0} surveys match to unique participant usernames".format(matched_usernames))

330 total surveys
325 surveys match to unique participant usernames


### Merge Surveys with Observational Data

In [19]:
for username, unit in units.items():
    if username in survey_rows.keys():
        survey = survey_rows[username]
        unit['completed_survey'] = True
        for key,value in survey_colnames.items():
            unit[value] = survey[key]
    else:
        unit['completed_survey'] = False
        for key, value in survey_colnames.items():
            unit[value] = None

# Count the Number of Days that an account was banned or muted during the observation period


In [20]:
for mod_action in participant_modlog:
    ## used to include mute actions, 'muteuser', 'unmuteuser'
    ## but these turned out not to be useful for the study
    ## and were not specified in the pre-analysis plan
    if mod_action.action in ['banuser', 'unbanuser']:
        username = mod_action.target_author
        units[username]['ban_actions'].append(mod_action)
Counter([len(x['ban_actions']) for x in units.values()])

Counter({0: 1362, 1: 263, 2: 54, 3: 5, 4: 1, 5: 1})

In [21]:
# an account is considered banned or muted during
# a given day if they were banned or muted at all
# during that 24 hour period

#test_units = [x for x in units.values() if len(x['ban_actions'])>0]

for unit in units.values(): #units.values():
    ban_status = False
    ban_datetime = None
    next_ban_pointer = 0
    
    observation_days = [0] * 8*7
    
    ## ASSIGN OBSERVATION DAYS
    for i in list(range(0, len(observation_days))):
        obs_date = unit['assignment_datetime'] + datetime.timedelta(days=i)

        if(next_ban_pointer < len(unit['ban_actions']) and 
           unit['ban_actions'][next_ban_pointer].created_utc <= obs_date):
            
            ban_action = unit['ban_actions'][next_ban_pointer]
            
            ## these previously included bans and mutes, but 
            ## mutes are not relevant for this study
            if(ban_action.action in ['banuser']):
                ban_status = True
            elif(ban_action.action in ['unbanuser']):
                ban_status = False
            
            next_ban_pointer += 1
            
        observation_days[i] = ban_status
    
    unit['ban_observation_days'] = observation_days
    
    ## RESET BAN DAY COUNTS
    for weeks in [2,4,8]:
        key = "ban_days_{0}_weeks".format(weeks)
        unit[key] = 0
        
    ## AGGREGATE OBSERVATION DAYS INTO COUNTS FOR EACH WEEK DURATION
    for i in list(range(0, len(observation_days))):
        for weeks in [2,4,8]:
            if i < weeks*7 and observation_days[i] == True:
                key = "ban_days_{0}_weeks".format(weeks)
                unit[key] += 1
        

# Create Account Mapping between usernames and unique IDs before outputting data

In [22]:
account_mapping = {}
for username in units.keys():
    account_mapping[username] = {
        "username": username,
        "uuid": uuid.uuid4()
    }

In [23]:
for username, unit in units.items():
    unit['id'] = str(account_mapping[username]['uuid'])
    unit['ban_observation_days'] = None
    unit['comments'] = None
    unit['ban_actions'] = None
    unit['username']  = None
    unit['comment_id'] = None

# Write to Files

In [24]:
pd.DataFrame(list(account_mapping.values())).to_csv(
    os.path.join(data_dir, "r-feminism-account-mapping-{0}.csv".format(datetime.datetime.utcnow().strftime('%m.%d.%Y'))))

In [25]:
pd.DataFrame(list(units.values())).to_csv(
    os.path.join(data_dir, "r-feminism-study-data-merged-{0}.csv".format(datetime.datetime.utcnow().strftime('%m.%d.%Y'))))